In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.model_selection import train_test_split, GridSearchCV, \
                                    ShuffleSplit, KFold

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.preprocessing import MinMaxScaler, StandardScaler, \
                                  RobustScaler, PolynomialFeatures, \
                                  OrdinalEncoder, LabelEncoder, \
                                  OneHotEncoder, TargetEncoder, \
                                  QuantileTransformer, PowerTransformer, \
                                  KBinsDiscretizer, FunctionTransformer

from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, RANSACRegressor, \
                                 Ridge, Lasso, LinearRegression

from sklearn.metrics import recall_score, precision_score, \
                            f1_score, ConfusionMatrixDisplay, \
                            confusion_matrix, roc_auc_score, \
                            RocCurveDisplay, PrecisionRecallDisplay, \
                            roc_curve, precision_recall_curve, \
                            PrecisionRecallDisplay, make_scorer, \
                            mean_squared_error, mean_squared_log_error
from sklearn.compose import ColumnTransformer, make_column_selector,\
                            make_column_transformer, TransformedTargetRegressor
from sklearn.metrics import fbeta_score
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
import pickle
import numba
import category_encoders as ce

In [2]:
import catboost
from catboost import CatBoostRegressor
print(catboost.__version__)

1.2.7


In [3]:
import math

In [4]:
import numpy as np
import pandas as pd
import optuna
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings

warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')

In [6]:
display(train.shape)
display(train.info)
display(train.head(5))

(1200000, 20)

<bound method DataFrame.info of           Age  Gender  Annual Income Marital Status  Number of Dependents  \
id                                                                          
0        19.0  Female        10049.0        Married                   1.0   
1        39.0  Female        31678.0       Divorced                   3.0   
2        23.0    Male        25602.0       Divorced                   3.0   
3        21.0    Male       141855.0        Married                   2.0   
4        21.0    Male        39651.0         Single                   1.0   
...       ...     ...            ...            ...                   ...   
1199995  36.0  Female        27316.0        Married                   0.0   
1199996  54.0    Male        35786.0       Divorced                   NaN   
1199997  19.0    Male        51884.0       Divorced                   0.0   
1199998  55.0    Male            NaN         Single                   1.0   
1199999  21.0  Female            NaN       D

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [7]:
display(train.dtypes)

Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age             float64
Credit Score            float64
Insurance Duration      float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
Premium Amount          float64
dtype: object

In [8]:
train.columns = train.columns.str.lower().str.replace(' ', '_')
test.columns = test.columns.str.lower().str.replace(' ', '_')

In [9]:
train.describe()

,age,annual_income,number_of_dependents,health_score,previous_claims,vehicle_age,credit_score,insurance_duration,premium_amount
count,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


# Functions to work with features

In [10]:

def feature_transformer(df):

    df['policy_start_date'] = pd.to_datetime(df['policy_start_date'])

    df['year'] = df['policy_start_date'].dt.year
    df['day'] = df['policy_start_date'].dt.day
    df['month'] = df['policy_start_date'].dt.month
    df['day_of_year'] = df['policy_start_date'].dt.dayofyear
    df['day_of_week'] = df['policy_start_date'].dt.weekday
    df['sin_day_of_week'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['cos_day_of_week'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    df["seconds since 1970"] = df['policy_start_date'].astype("int64") // 10**9
    
    df['days_passed'] = (df['policy_start_date'].max() - df['policy_start_date']).dt.days
    
    df['cat_day_of_week'] = df['day_of_week'].astype('string')
    df['cat_year'] = df['year'].astype('string')
    df['cat_day'] = df['day'].astype('string')
    df['cat_day_of_year'] = df['day_of_year'].astype('string')
    df['cat_month'] = df['month'].astype('string')
    

    policy_starts_min = df['policy_start_date'].min()  # 2019-08-17
    year = policy_starts_min.year  
    
    if policy_starts_min >= pd.Timestamp(f'{year}-01-01'):
        fiscal_year_start = pd.Timestamp(f'{year}-08-01')
    else:
        fiscal_year_start = pd.Timestamp(f'{year-1}-08-01')

    
    df['time_from_fiscal_year'] = (df['policy_start_date'] - fiscal_year_start).dt.days
    df['seconds_from_fiscal_year'] = (df['policy_start_date'] - fiscal_year_start).dt.total_seconds()

    new_date = policy_starts_min - timedelta(days=1)
    
    df['time_from_first_policy'] = (df['policy_start_date'] - new_date).dt.days
    
    df['time_from_first_policy_seconds'] = (df['policy_start_date'] - new_date).dt.total_seconds()
    
    df['Days Passed'] = (df['policy_start_date'].max() - df['policy_start_date']).dt.days

    df['claims_vs_duration'] = df['previous_claims'] / df['insurance_duration']
    df['days_from_2019_crisis'] = (df['policy_start_date'] - pd.Timestamp('2019-01-01')).dt.days
    df['revenue_per_dependent'] = (df['annual_income'] / df['number_of_dependents'] + 1)
    df['ratio_of_doubts'] = (df['previous_claims'] + 1) / df['annual_income'] # NEW
    
    df['marital_status_customer_feedback'] = df['marital_status'] + df['customer_feedback']
    df['customer_feedback_property_type'] = df['customer_feedback'] + df['property_type']
    df['customer_feedback_year'] = df['customer_feedback'] + df['cat_year'] # тут упала
    df['marital_status_year'] = df['marital_status'] + df['cat_year']
    df['exercise_frequency_year'] = df['exercise_frequency'] + df['cat_year']
    df['customer_feedback_smoking_status'] = df['customer_feedback'] + df['smoking_status']
    df['customer_feedback_property_type'] = df['customer_feedback'] + df['property_type']
    df['year_day_of_week'] = df['cat_year'] + df['cat_day_of_week']
    df['property_type_year'] = df['property_type'] + df['cat_year']
    df['policy_type_year'] = df['policy_type'] + df['cat_year']
    df['gender_marital_status'] = df['gender'] + df['marital_status']
    df['education_level_customer_feedback'] = df['education_level'] + df['customer_feedback']
    df['gender_year'] = df['gender'] + df['cat_year']
    df['marital_status_day_of_week'] = df['marital_status'] + df['cat_day_of_week']
    df['occupation_customer_feedback'] = df['occupation'] + df['customer_feedback']
    df['occupation_year'] = df['occupation'] + df['cat_year']
    df['location_customer_feedback'] = df['location'] + df['customer_feedback'] 
    df['location_day_of_week'] = df['location'] + df['cat_day_of_week']
    df['education_level_year'] = df['education_level'] + df['cat_year']
    df['customer_feedback_month'] = df['customer_feedback'] + df['cat_month']
    df['marital_status_day'] = df['marital_status'] + df['cat_day']
    df['exercise_frequency_property_type'] = df['exercise_frequency'] + df['property_type']
    df['education_level_day_of_week'] = df['education_level'] + df['cat_day_of_week']

    df.drop(columns=['time_from_first_policy', 'policy_start_date'], inplace=True)

    return df

In [11]:
# заполнение na нулями в датафрейме
def columns_imputer(df):
    for c in df.columns:
        df[f"is_{c}_na"] = df[c].isna().astype(int)

    return df

In [12]:
# столбцы с цифрами и нули в них
num_cols = ['age', 'annual_income', 'number_of_dependents', 'health_score','previous_claims', 'vehicle_age', 'credit_score', 'insurance_duration']

In [13]:
# цифры в категории
def num_to_cat(df):
    global num_cols

    for col in num_cols:
        df[f"categorial_{col}"] = df[col].astype('category')

    return df

In [14]:
def data_get(X):
    X = feature_transformer(X)
    X = columns_imputer(X)
    X = num_to_cat(X)
    cats = X.select_dtypes(include=['string', 'category', 'object']).columns.tolist()
    nums = X.select_dtypes(include=['number']).columns.tolist()
    for col in cats:
        X[col] = X[col].astype('string')
    X[cats] = X[cats].fillna('None').astype('string')
    X[nums] = X[nums].fillna(-999).astype(float)
    return X

In [15]:
X = train.drop('premium_amount', axis=1)
y = train['premium_amount']

X = data_get(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=13)

In [17]:
catboost_params = {'iterations': 800, 'learning_rate': 0.05, 'depth': 8, 'l2_leaf_reg': 3, 'loss_function': 'RMSE', 'random_strength': 0.05, 'bagging_temperature': 0.01}

In [18]:
X_test = test
X_test = data_get(X_test)

In [21]:
class RMSLE(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        result = []
        for index in range(len(targets)):
            val = max(approxes[index], 0)
            der1 = math.log1p(targets[index]) - math.log1p(max(0, approxes[index]))
            der2 = -1 / (max(0, approxes[index]) + 1)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result
class RMSLE_val(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * ((math.log1p(max(0, approx[i])) - math.log1p(max(0, target[i])))**2)

        return error_sum, weight_sum

In [26]:
cat_features = X.select_dtypes(exclude=['number']).columns.tolist()

In [27]:
%%time

params_2 = {'loss_function':RMSLE(),#'Quantile:alpha=0.41',
          'eval_metric':RMSLE_val(),
          #'task_type': 'GPU',
          'verbose': 200,
          'random_seed': 13,
          #'grow_policy': 'Lossguide',
          'l2_leaf_reg': 3,
          'depth': 8,
          'learning_rate': 0.05,
          'iterations': 800,
          'random_strength': 0.05,
          'bagging_temperature': 0.01
         }
cbr_1 = CatBoostRegressor(**params_2)
cbr_1.fit(X_train, y_train, cat_features=cat_features, 
          use_best_model=True,
          plot=True
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 6.4025627	total: 1.84s	remaining: 24m 28s
200:	learn: 1.0432806	total: 5m 5s	remaining: 15m 9s
400:	learn: 1.0389126	total: 12m 15s	remaining: 12m 11s
600:	learn: 1.0376836	total: 19m 45s	remaining: 6m 32s
799:	learn: 1.0373328	total: 25m 22s	remaining: 0us
CPU times: user 2h 35min 19s, sys: 2min 11s, total: 2h 37min 31s
Wall time: 25min 30s


In [42]:
def get_preds(cbr_1, X_test):
    test_predict = pd.DataFrame(cbr_1.predict(X_test), columns=['Premium Amount'], index=X_test.index)
    test_predict.index.names = ['id']
    display(test_predict.head(5))
    display(test_predict.index.names)
    display(test_predict.shape)
    test_predict.to_csv('test_preds_10.csv')

In [43]:
get_preds(cbr_1, X_test)

,Premium Amount
id,
1200000,838.946557
1200001,867.625191
1200002,847.026731
1200003,783.181291
1200004,732.412305


FrozenList(['id'])

(800000, 1)

In [39]:
# def get_preds(preds, data_test):
#     test_predict = pd.DataFrame(test_preds, columns=['Premium Amount'], index=X_test.index)
#     test_predict.index.names = ['id']
#     display(test_predict.head(5))
#     display(test_predict.index.names)
#     display(test_predict.shape)
#     test_predict.to_csv('test_preds_9.csv')

In [40]:
# get_preds(test_preds, X_test)

### Final

In [ ]:
def feature_transformer(df):

    df['policy_start_date'] = pd.to_datetime(df['policy_start_date'])

    df['year'] = df['policy_start_date'].dt.year
    df['day'] = df['policy_start_date'].dt.day
    df['month'] = df['policy_start_date'].dt.month
    df['day_of_year'] = df['policy_start_date'].dt.dayofyear
    df['day_of_week'] = df['policy_start_date'].dt.weekday
    df['sin_day_of_week'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['cos_day_of_week'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    df["seconds since 1970"] = df['policy_start_date'].astype("int64") // 10**9
    
    df['days_passed'] = (df['policy_start_date'].max() - df['policy_start_date']).dt.days
    
    df['cat_day_of_week'] = df['day_of_week'].astype('string')
    df['cat_year'] = df['year'].astype('string')
    df['cat_day'] = df['day'].astype('string')
    df['cat_day_of_year'] = df['day_of_year'].astype('string')
    df['cat_month'] = df['month'].astype('string')
    

    policy_starts_min = df['policy_start_date'].min()  # 2019-08-17
    year = policy_starts_min.year  
    
    if policy_starts_min >= pd.Timestamp(f'{year}-01-01'):
        fiscal_year_start = pd.Timestamp(f'{year}-08-01')
    else:
        fiscal_year_start = pd.Timestamp(f'{year-1}-08-01')

    
    df['time_from_fiscal_year'] = (df['policy_start_date'] - fiscal_year_start).dt.days
    df['seconds_from_fiscal_year'] = (df['policy_start_date'] - fiscal_year_start).dt.total_seconds()

    new_date = policy_starts_min - timedelta(days=1)
    
    df['time_from_first_policy'] = (df['policy_start_date'] - new_date).dt.days
    
    df['time_from_first_policy_seconds'] = (df['policy_start_date'] - new_date).dt.total_seconds()
    
    df['Days Passed'] = (df['policy_start_date'].max() - df['policy_start_date']).dt.days

    df['claims_vs_duration'] = df['previous_claims'] / df['insurance_duration']
    df['days_from_2019_crisis'] = (df['policy_start_date'] - pd.Timestamp('2019-01-01')).dt.days
    df['revenue_per_dependent'] = (df['annual_income'] / df['number_of_dependents'] + 1)
    df['ratio_of_doubts'] = (df['previous_claims'] + 1) / df['annual_income'] # NEW
    
    df['marital_status_customer_feedback'] = df['marital_status'] + df['customer_feedback']
    df['customer_feedback_property_type'] = df['customer_feedback'] + df['property_type']
    df['customer_feedback_year'] = df['customer_feedback'] + df['cat_year'] # тут упала
    df['marital_status_year'] = df['marital_status'] + df['cat_year']
    df['exercise_frequency_year'] = df['exercise_frequency'] + df['cat_year']
    df['customer_feedback_smoking_status'] = df['customer_feedback'] + df['smoking_status']
    df['customer_feedback_property_type'] = df['customer_feedback'] + df['property_type']
    df['year_day_of_week'] = df['cat_year'] + df['cat_day_of_week']
    df['property_type_year'] = df['property_type'] + df['cat_year']
    df['policy_type_year'] = df['policy_type'] + df['cat_year']
    df['gender_marital_status'] = df['gender'] + df['marital_status']
    df['education_level_customer_feedback'] = df['education_level'] + df['customer_feedback']
    df['gender_year'] = df['gender'] + df['cat_year']
    df['marital_status_day_of_week'] = df['marital_status'] + df['cat_day_of_week']
    df['occupation_customer_feedback'] = df['occupation'] + df['customer_feedback']
    df['occupation_year'] = df['occupation'] + df['cat_year']
    df['location_customer_feedback'] = df['location'] + df['customer_feedback'] 
    df['location_day_of_week'] = df['location'] + df['cat_day_of_week']
    df['education_level_year'] = df['education_level'] + df['cat_year']
    df['customer_feedback_month'] = df['customer_feedback'] + df['cat_month']
    df['marital_status_day'] = df['marital_status'] + df['cat_day']
    df['exercise_frequency_property_type'] = df['exercise_frequency'] + df['property_type']
    df['education_level_day_of_week'] = df['education_level'] + df['cat_day_of_week']

    df.drop(columns=['time_from_first_policy', 'policy_start_date'], inplace=True)

    return df

In [ ]:
class RMSLE(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        result = []
        for index in range(len(targets)):
            val = max(approxes[index], 0)
            der1 = math.log1p(targets[index]) - math.log1p(max(0, approxes[index]))
            der2 = -1 / (max(0, approxes[index]) + 1)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result
class RMSLE_val(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * ((math.log1p(max(0, approx[i])) - math.log1p(max(0, target[i])))**2)

        return error_sum, weight_sum

In [ ]:
ct_time = ColumnTransformer([
     ("hour",  transform_time('hour'),['Policy Start Date']),
     ("week",  transform_time('weekday'),['Policy Start Date']),
     ("month", transform_time('month'),['Policy Start Date']),
     ("year", transform_time('year'),['Policy Start Date'])
    ],remainder='drop').set_output(transform='pandas')


CT_sin_cos = ColumnTransformer([
    # ('month_sin',sin_transformer(12),['month__Policy Start Date']),
    # ('month_cos',cos_transformer(12),['month__Policy Start Date']),
    # ('week_sin',sin_transformer(7),['week__Policy Start Date']),
    # ('week_cos',cos_transformer(7),['week__Policy Start Date']),
    # ('hour_sin',sin_transformer(24),['hour__Policy Start Date']),
    # ('hour_cos',cos_transformer(24),['hour__Policy Start Date']),
    ('year_encoder',OneHotEncoder(sparse_output=False),['year__Policy Start Date'])
]).set_output(transform='pandas')


time_pipe=Pipeline(steps=[
    ('time',ct_time),
    ('sin_cos',CT_sin_cos)
])

In [ ]:
nums = Pipeline(steps=[
   # ('missing_nums',IterativeImputer(max_iter=20)), #проверить, дропнул ли na в eda, если да, то бесполезный код
    #('bins', KBinsDiscretizer(n_bins=10, strategy='uniform', encode='ordinal')), #делаем бинаризацию
    #('polynom', PolynomialFeatures(3, include_bias=False)), #делаем полиномы, если надо
    ('scaler', StandardScaler())])

annual_income_pipe= Pipeline(steps=[
    ('missing_num',IterativeImputer(max_iter=20)),
    ('transformer',PowerTransformer())
])
credit_score_pipe = Pipeline(steps=[
    ('missing_num',IterativeImputer(max_iter=20)),
    ('FU', FeatureUnion([
        ('origin', 'passthrough'),
        ('kbin', KBinsDiscretizer(n_bins=10, strategy='uniform', encode='onehot-dense'))])),
    ('poly',PolynomialFeatures((1,2),interaction_only=True,include_bias=False))
     ])

clarity_map = [{
    'col':'Education Level',
    'mapping': educ_mapping
    },
    {'col': 'Occupation',
    'mapping':occup_mapping
    },
    {'col': 'Location',
    'mapping':loc_mapping
    },
    {'col': 'Policy Type',
    'mapping':pt_mapping
    },
    {'col': 'Customer Feedback',
    'mapping':cf_mapping
    },
    {'col': 'Exercise Frequency',
    'mapping':ef_mapping
    },
    {'col': 'Property Type',
    'mapping':pt_mapping
    },
    {'col': 'Marital Status',
    'mapping':ms_mapping
    }]

special_transformer = Pipeline(steps=[
    ('missing_nums', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', MinMaxScaler())
    ])    
    
CT = ColumnTransformer([
        ("with_nums", nums, numerical_features),
        ("with_category", OneHotEncoder(sparse_output=False, handle_unknown='error'), category_features), #если не одна категория, то лучше прописать параметр drop
        ("special_map", special_transformer, spec_m),
        ('time',time_pipe,['Policy Start Date']),
        ("credit_score", credit_score_pipe, ['Credit Score']), 
        ('annual_income',annual_income_pipe,['Annual Income'])
        ]).set_output(transform='pandas')


display(CT)

ct_train = CT.fit_transform(data_train)
ct_valid = CT.transform(data_valid)
pd.DataFrame(ct_train).head(10).T


In [ ]:
catboost_pipeline = Pipeline(steps=[
    ('preprocessing', CT), #забиваем свой препроцессор, если надо, можно написать в feature engineering universal_preprocessor
    ('model', CatBoostRegressor())]) #забиваем свою модель, если надо
display(catboost_pipeline)